In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
file_path = "bidmc_final_merged.csv"
df = pd.read_csv(file_path)

In [14]:
df.columns = df.columns.str.strip()
df["Disease_Label"] = ((df["SpO2"] < 92) | (df["RESP_x"] < 12) | (df["RESP_x"] > 20)).astype(int)

In [15]:
features = ["HR", "PULSE", "RESP_x", "SpO2", "PLETH", "II"]  # Adjusted feature set
target = "Disease_Label"


In [16]:
df = df.dropna(subset=features + [target])

In [17]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

In [18]:
time_steps = 10
X, y = [], []
for i in range(len(X_scaled) - time_steps):
    X.append(X_scaled[i : i + time_steps]) 
    y.append(df[target].iloc[i + time_steps])

In [19]:
X, y = np.array(X), np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, len(features))),
    MaxPooling1D(pool_size=2),
    LSTM(50, return_sequences=True),
    LSTM(50),
    Dense(25, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

c:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 25s 19ms/step - accuracy: 0.9014 - loss: 0.2880 - val_accuracy: 0.9601 - val_loss: 0.1134
Epoch 2/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.9621 - loss: 0.1125 - val_accuracy: 0.9712 - val_loss: 0.1049
Epoch 3/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.9738 - loss: 0.0824 - val_accuracy: 0.9747 - val_loss: 0.0741
Epoch 4/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.9796 - loss: 0.0667 - val_accuracy: 0.9757 - val_loss: 0.0743
Epoch 5/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.9787 - loss: 0.0695 - val_accuracy: 0.9706 - val_loss: 0.0860
Epoch 6/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.9797 - loss: 0.0694 - val_accuracy: 0.9781 - val_loss: 0.0678
Epoch 7/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9820 - loss: 0.0624 - val_accuracy: 0.9781 - val_loss: 0.0701
Epoch 8/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.9832 - loss: 0.0591 - val_

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy:.2f}")


146/146 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9809 - loss: 0.0704
Model Accuracy: 0.98


In [23]:
model.save("/mnt/data/cnn_lstm_model.h5")
print("🎉 Model saved as cnn_lstm_model.h5")

🎉 Model saved as cnn_lstm_model.h5
